<a href="https://colab.research.google.com/github/MattJCR/Introduccion-Whisper-tratado-correccion-analisis/blob/master/Introducci%C3%B3n_a_Whisper%2C_tratado%2C_correcci%C3%B3n_y_an%C3%A1lisis_de_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# !sudo apt update && sudo apt install ffmpeg
!pip install --upgrade git+https://github.com/openai/whisper.git pytube pydub pyspellchecker collection nltk
# !pip install --upgrade pytube pydub pyspellchecker collection
# !pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-8r7n01hk
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-8r7n01hk


In [62]:
# Import the required packages
from pytube import YouTube
# import ffmpeg
import whisper
import numpy as np
import pandas as pd
from spellchecker import SpellChecker
import re
from collections import Counter
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('universal_tagset')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [63]:

# download the video using pytube
yt = YouTube("https://www.youtube.com/watch?v=AHPrLLrCa7Y")
video = yt.streams.first().download()
video.title()

'/Content/Todo Harry Potter  Resumen En 1 Minuto.3Gpp'

In [64]:
for x in whisper.available_models(): print(x) 

tiny.en
tiny
base.en
base
small.en
small
medium.en
medium
large-v1
large-v2
large


In [65]:
# load the downloaded video using whisper
audio = whisper.load_audio(video)

# transcribe the audio using whisper's medium model
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} ",
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)



Model is multilingual  and has 762,321,920 parameters.


In [66]:
# load audio and pad/trim it to fit 30 seconds
audio_part = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio_part).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
language = max(probs, key=probs.get)
print(f"Detected language: {language}")

# load the spelling corrector
corrector = SpellChecker(language=language)

Detected language: es


In [67]:
result = model.transcribe(audio=audio)

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [116]:
correction_text = ''
# print the recognized text
for text in result['text'].split('.'):
  # correct the spelling of the mispelled words
  text += '.'
  correction = corrector.correction(text)
  if(correction):
    print('-'*20)
    correction_text += correction
    print('Ori:',text,'\nCor:',correction)

--------------------
Ori:  Ocho películas de Harry Potter, una saga en un minuto, pero lo voy a conseguir. 
Cor:  Ocho películas de Harry Potter, una saga en un minuto, pero lo voy a conseguir.
--------------------
Ori:  Advertencia, este vídeo contiene spoilers. 
Cor:  Advertencia, este vídeo contiene spoilers.
--------------------
Ori:  Si eres alérgico a spoilers o crees que puede serlo, acude a tu sala de cine más cercana. 
Cor:  Si eres alérgico a spoilers o crees que puede serlo, acude a tu sala de cine más cercana.
--------------------
Ori:  Harry Potter es un niño normal que un buen día recibe una carta para ir a Hogwarts. 
Cor:  Harry Potter es un niño normal que un buen día recibe una carta para ir a Hogwarts.
--------------------
Ori:  ¿Y qué es Hogwarts? Es un colegio de magia supermolol. 
Cor:  ¿Y qué es Hogwarts? Es un colegio de magia supermolol.
--------------------
Ori:  Tiene perros gigantes de tres cabezas, de mentores que te absorben el alma y un enemigo mortal llam

In [117]:
import string
def process_text(text):
    # remove punctuation and special characters
    processed_text = text.translate(str.maketrans('', '', string.punctuation + '¿¡'))
    # convert to lowercase
    processed_text = processed_text.lower()
    return processed_text
    
def count_most_common_words(processed_text):
    # split the text into a list of words
    words = processed_text.split()
    # create a Counter object
    word_counts = Counter(words)
    # return the most common words in descending order of frequency
    return word_counts.most_common()

In [121]:
# count the most common words
correction_text = process_text(correction_text)
most_common_words = count_most_common_words(correction_text)
# print the results
print("The most common words are:")
counter = 1
for word, count in most_common_words:
    print(f"{counter}) {word}: {count}")
    if(counter >= 5):
      break
    else:
      counter += 1

The most common words are:
1) que: 15
2) un: 12
3) a: 11
4) de: 9
5) y: 9


In [122]:
correction_text

' ocho películas de harry potter una saga en un minuto pero lo voy a conseguir advertencia este vídeo contiene spoilers si eres alérgico a spoilers o crees que puede serlo acude a tu sala de cine más cercana harry potter es un niño normal que un buen día recibe una carta para ir a hogwarts y qué es hogwarts es un colegio de magia supermolol tiene perros gigantes de tres cabezas de mentores que te absorben el alma y un enemigo mortal llamado voldemort que solo te ataca el último trimestre de cada curso para que haga los examenes más tranquilos a harry no le está yendo nada mal en el cole con 11 años se carga un profesor tartabudo con 12 lucha contra un basilisco gigante y con 13 ayuda a escapar de la justicia a un fugitivo condenado por el asesinato de 20 personas oye seguro que este chico es el bueno y qué piensa sus padres a que no tiene bueno el caso es que todo cambia en cuarto curso cuando harry descubre que voldemort ha vuelto y que ha matado a cedric dibory no por qué ahora se co

In [123]:
# Load Spanish part-of-speech tagger from nltk
nltk.download('cess_esp')
tagger = nltk.tag.UnigramTagger(nltk.corpus.cess_esp.tagged_sents())

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


In [124]:
# Define a function to tag a sentence with its part-of-speech
def tag_sentence(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged_tokens = tagger.tag(tokens)
    return tagged_tokens

# Define a function to extract nouns, adjectives, and verbs from a tagged sentence
def extract_pos(tagged_sentence):
    nouns = []
    adjectives = []
    verbs = []
    others = []
    for word, pos in tagged_sentence:
      if pos != None:
        if pos[0] == 'n':
            nouns.append(word)
        elif pos[0] == 'a':
            adjectives.append(word)
        elif pos[0] == 'v':
            verbs.append(word)
        else:
            others.append(word)
    return nouns, adjectives, verbs, others

# Define the sentence to tag and extract POS
tagged_sentence = tag_sentence(correction_text)
nouns, adjectives, verbs, others = extract_pos(tagged_sentence)
# Print the results
print(f"Nouns: {nouns}")
print(f"Adjectives: {adjectives}")
print(f"Verbs: {verbs}")
# print(f"Others: {others}")

Nouns: ['películas', 'saga', 'minuto', 'advertencia', 'vídeo', 'cine', 'niño', 'día', 'carta', 'colegio', 'magia', 'perros', 'gigantes', 'cabezas', 'alma', 'enemigo', 'trimestre', 'curso', 'años', 'carga', 'profesor', 'lucha', 'gigante', 'ayuda', 'justicia', 'asesinato', 'personas', 'chico', 'padres', 'caso', 'curso', 'luz', 'sol', 'práctica', 'chaval', 'años', 'duelo', 'condiciones', 'edad', 'colegio', 'vida', 'cursos', 'parte', 'fama', 'amigos', 'novia']
Adjectives: ['cercana', 'normal', 'buen', 'mortal', 'llamado', 'solo', 'último', 'seguro', 'bueno', 'bueno', 'cuarto', 'octava', 'suficiente']
Verbs: ['voy', 'conseguir', 'contiene', 'eres', 'puede', 'acude', 'es', 'recibe', 'ir', 'es', 'es', 'tiene', 'haga', 'está', 'escapar', 'condenado', 'oye', 'es', 'piensa', 'tiene', 'es', 'cambia', 'descubre', 'ha', 'vuelto', 'ha', 'matado', 'convertirá', 've', 'tiene', 'convirtiendo', 'acabando', 'va', 'conseguir', 'tendrá', 'abandonar', 'ha', 'intentado', 'acabar', 'ha', 'dado']
